In [55]:
import pandas as pd
import requests
import json
import datetime

In [56]:
gare = "Paris Montparnasse Hall 1 & 2"

# Max 250
nb_trains = 20

In [57]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

## Dictionnaire

In [58]:
req_gare = requests.get("https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&rows=3220&sort=gare_alias_libelle_noncontraint&facet=departement_libellemin&facet=segmentdrg_libelle&facet=gare_agencegc_libelle&facet=gare_regionsncf_libelle&facet=gare_ug_libelle")
doc_gare = json.loads(req_gare.text)
row_gare = len(doc_gare['records'])
print(f'Nombre de lignes : {row_gare}')

Nombre de lignes : 3220


In [59]:
df_dic = pd.DataFrame(doc_gare['records'])
df_dic = pd.DataFrame(list(df_dic['fields']))
df_dic = df_dic[['alias_libelle_noncontraint','uic_code']]

In [60]:
def removezero(string):
    return 'SNCF:' + str(string)[2:]

df_dic['uic_code'] = df_dic['uic_code'].apply(removezero)

In [61]:
df_dic.convert_dtypes()
dic_gare = df_dic.set_index('alias_libelle_noncontraint').T.to_dict('list')

/var/folders/bg/vbvjjnmj0hv9wmb92ngxcq9m0000gn/T/ipykernel_6067/3118890846.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dic_gare = df_dic.set_index('alias_libelle_noncontraint').T.to_dict('list')


## Début

In [62]:
link = 'https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:' + dic_gare[gare][0] + '/arrivals?count=' + str(nb_trains)
req = requests.get(link,auth=(token, ''))
#print(link)

In [63]:
doc = json.loads(req.text)
row = len(doc['arrivals'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 20


In [64]:
df = pd.DataFrame(doc['arrivals'])
df_gare = pd.DataFrame(list(df['display_informations']))
df_heure = pd.DataFrame(list(df['stop_date_time']))
df_id = pd.DataFrame(list(df['links']))
df_id = pd.DataFrame(list(df_id[1]))

In [65]:
def del_day(string):
    return string[9:]

df_heure['arrival_date_time'] = df_heure['arrival_date_time'].apply(del_day)
df_heure['base_arrival_date_time'] = df_heure['base_arrival_date_time'].apply(del_day)

In [66]:
def conv_min(string):
    return int(string[0:2])*60 + int(string[2:4])

df_heure['retard'] = df_heure['arrival_date_time'].apply(conv_min) - df_heure['base_arrival_date_time'].apply(conv_min)

In [67]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_heure['heure'] = df_heure['arrival_date_time'].apply(str_tps)
df_heure['old_heure'] = df_heure['base_arrival_date_time'].apply(str_tps)

In [68]:
def del_par(string):
    index = string.find("(")
    return string[:index]

df_gare['direction'] = df_gare['direction'].apply(del_par)

In [69]:
df_gare = df_gare[['direction','network','trip_short_name']]
df_gare.rename(columns = {'direction':'Destination'}, inplace = True)
df_gare.rename(columns = {'network':'Train'}, inplace = True)
df_gare.rename(columns = {'trip_short_name':'Numéro'}, inplace = True)

df_gare['Arrivée (réelle)'] = df_heure['heure']
df_gare['Arrivée (prévue)'] = df_heure['old_heure']
df_gare['Retard (min)'] = df_heure['retard']
df_gare['id'] = df_id['id']

In [70]:
def get_name(string):
    string = string[10:]
    index_fin = string.find("', 'links'")
    return string[:index_fin]

arrets = []
causes = []

for i in df_gare.index:
    id = df_gare.iloc[i]['id']
    
    if 'RealTime' in id:
        index_id = id.index("RealTime")
        id = id[:index_id-1]

    link_voyage = 'https://api.sncf.com/v1/coverage/sncf/vehicle_journeys/' + id
    req_arret = requests.get(link_voyage ,auth=(token, ''))
    doc_voyage = json.loads(req_arret.text)

    df_arret = pd.DataFrame(doc_voyage['vehicle_journeys'])
    df_arret = pd.DataFrame(list(df_arret['stop_times']))
    df_arret = df_arret.T
    df_arret = pd.DataFrame(list(df_arret[0]))
    df_arret['stop_point'] = df_arret['stop_point'].astype('str').apply(get_name)

    if df_gare.iloc[i]['Retard (min)'] != 0:
        df_retard = pd.DataFrame(doc_voyage['disruptions'])
        if 'messages' in df_retard:
            df_retard = pd.DataFrame(list(df_retard['messages'][0]))
            causes.append(df_retard.iloc[0]['text'])
        else:
            causes.append("Retard non expliqué")
    else:
        causes.append("")

    liste_arrets = list(df_arret['stop_point'])

    arrets.append(liste_arrets)

In [71]:
df_gare['Cause'] = causes
df_gare['Arrêts'] = arrets
df_gare = df_gare.drop(['id'], axis=1)

In [72]:
df_gare

,Destination,Train,Numéro,Arrivée (réelle),Arrivée (prévue),Retard (min),Cause,Arrêts
0,Paris - Montparnasse - Hall 1 & 2,TGV INOUI,8583,17:52:00,17:52:00,0,,"[Tarbes, Lourdes, Pau, Orthez, Dax, Bordeaux S..."
1,Paris - Montparnasse - Hall 1 & 2,TGV INOUI,8540,17:52:00,17:52:00,0,,"[Hendaye, Saint-Jean-de-Luz - Ciboure, Biarrit..."
2,Paris - Montparnasse - Hall 1 & 2,TER,16774,17:53:00,17:53:00,0,,"[Le Mans, Connerré - Beillé, La Ferté-Bernard,..."
3,Paris - Montparnasse - Hall 1 & 2,TRANSILIEN,165448,17:56:50,17:56:50,0,,"[Rambouillet, Le Perray, Les Essarts-le-Roi, C..."
4,Paris - Montparnasse - Hall 1 & 2,TRANSILIEN,165850,18:02:30,18:02:30,0,,"[Dreux, Marchezais - Broué, Houdan, Tacoignièr..."
5,Paris - Montparnasse - Hall 1 & 2,TGV INOUI,8630,18:08:30,18:08:30,0,,"[Brest, Morlaix, Plouaret - Trégor, Guingamp, ..."
6,Paris - Montparnasse - Hall 1 & 2,TRANSILIEN,165650,18:11:40,18:11:40,0,,"[Mantes-la-Jolie, Épône - Mézières-sur-Seine, ..."
7,Paris - Montparnasse - Hall 1 & 2,TRANSILIEN,165450,18:12:00,18:12:00,0,,"[Rambouillet, Saint-Quentin en Yvelines - Mont..."
8,Paris - Montparnasse - Hall 1 & 2,TGV INOUI,8880,18:25:00,18:25:00,0,,"[Nantes, Angers-Saint-Laud, Le Mans, Paris - M..."
9,Paris - Montparnasse - Hall 1 & 2,TRANSILIEN,165452,18:26:50,18:26:50,0,,"[Rambouillet, Le Perray, Les Essarts-le-Roi, C..."
